## Running Millar et al. (2021) R code to delineate storm events and determine hysteresis and flushing indicies.

In [2]:
# Main script to separate baseflow, delineate storm events, 
# and run c-Q hysteresis index analyses

# Clear memory
rm(list=ls())

#################
# LOAD PACKAGES #
#################

library(tidyverse)
library(viridis)

###################
# SET DIRECTORIES #
###################

# Define the input directory
source_dir <- "~//OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/"
input_dir <- "~//OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/sonya-may24/data"
output_dir <- "~//OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/Millar-WY23-single-events/output/"

#####################
# READ IN FUNCTIONS #
#####################

source(file.path(source_dir,"cQ_functions.R"))

In [37]:
################
# READ IN DATA #
################

# Downloaded on 05.27.24 from SV email
allInputData15Min <- read.csv(file.path(input_dir,"hb_q_chem copy.csv"))

# Filter the data for just Hungerford Brook for the year/time range you want
# Remove rows with missing values
Site_input <- allInputData15Min %>%
  #filter(year(datetime) == 2022) %>%
  drop_na(q_cms_hb, NO3) %>%
  select(datetime, q_cms = q_cms_hb, conc = NO3)

# Specify constituent in data set name
dataSetName <- "HF_NO3_WY23_AB"

# Chose constitution for plot axes labels (NO3, TOC, or turbidity)
constit <- "NO3"

Site_input$datetime <- ymd_hms(Site_input$datetime)

# Filter into just event
Site_input <- Site_input %>%
  filter(between(datetime, as.Date("2023-02-09"), as.Date("2023-02-16"))) 

# Rescale the data
Site_input <- Site_input %>% 
  mutate(rescaled_conc = ((conc-min(conc))/(max(conc)-min(conc))*max(q_cms)))

# Vector containing candidate baseflow separation filter values
candidateFilterPara <- c(0.96)

# Vector containing candidate stormflow threshold values
candidateSfThresh <- c(0.06, 0.04, 0.02) # MED HF guess values; Kincaid et al 2020 used 0.1 for Hungerford

# Vector with interpolation intervals used for calculating HI
interp <- seq(0,1,0.01)

In [38]:
##########################################
# RUN ANALYSIS TO GET HYSTERESIS INDICES #
##########################################

batchRun1 <- batchRunBfAndEvSepForCQ(qInputs = Site_input,
                                     bfSepPasses = 3,
                                     filterParam = candidateFilterPara,
                                     sfSmoothPasses = 4,
                                     sfThresh = candidateSfThresh,
                                     cInputs = Site_input,
                                     timeStep = 30,
                                     minDuration = 2,
                                     maxDuration = 200)

eventsDataAll1 <- getAllStormEvents(batchRun = batchRun1,
                                    timestep_min = 30)

batchRunFlowsLF1 <- batchRunflowCompare(qData = Site_input,
                                         bfSepPasses = 4,
                                         filterPara = candidateFilterPara,
                                         sfSmoothPasses = 4)

eventsData1 <- stormEventCalcs(batchRun = batchRun1,
                               timestep_min = 30)

stormCounts1 <- stormCounts(batchRun1)

hysteresisData1 <- getHysteresisIndices(batchRun = batchRun1,
                                        xForInterp = interp,
                                        eventsData = eventsData1)

Warning message in left_join(stormIds, conc, by = "datetime"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 210 of `x` matches multiple rows in `y`.
ℹ Row 210 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence
  this warning.”
Warning message in left_join(stormIds, conc, by = "datetime"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 210 of `x` matches multiple rows in `y`.
ℹ Row 210 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence
  this warning.”
Warning message in left_join(stormIds, conc, by = "datetime"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 210 of `x` matches multiple rows in `y`.
ℹ Row 210 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence
  t

In [39]:
######################
# EXPORT OUTPUT DATA #
######################

write.csv(eventsData1,file = file.path(output_dir,paste(dataSetName,"_StormEventSummaryData.csv",sep="")))
write.csv(batchRunFlowsLF1,file = file.path(output_dir,paste(dataSetName,"_DischargeData.csv",sep="")))
write.csv(hysteresisData1,file = file.path(output_dir,paste(dataSetName,"_HysteresisData.csv",sep="")))
write.csv(eventsDataAll1,file = file.path(output_dir,paste(dataSetName,"_AllCQData.csv",sep="")))
write.csv(stormCounts1,file = file.path(output_dir,paste(dataSetName,"_StormCounts.csv",sep="")))

In [42]:
#########################################
# PLOT AND SAVE DATA - EVENT SEPARATION #
#########################################

# Make subfolder in output directory to save hydrograph plots
dir.create(file.path(output_dir, "Hydrographs"), showWarnings = FALSE)

# 1) Plot and save the hydrograph with input data

initialHydrograph <- ggplot(Site_input,aes(x=datetime, y=q_cms)) +
                            geom_line(size=0.5, color="black") +
                            xlab(NULL) +
                            ylab(expression(paste("Total discharge (",m^3," ",s^-1,")"))) +
                            theme_bw() +
                            theme(text=element_text(size=18))

ggsave(file=file.path(output_dir,"Hydrographs",paste(dataSetName,"_TotalDischarge.jpeg")),
       initialHydrograph,
       width = 12, 
       height = 4, 
       units = "in",
       dpi=600)


# 2) Plot total discharge with baseflow

baseflowHydrograph <- ggplot() + 
                            geom_line(data=batchRunFlowsLF1, aes(x=datetime, y=total_flow), size=0.5, color="black") +
                            geom_line(data=batchRunFlowsLF1, aes(x=datetime, y=base_flow,color=filter_para), size=0.75) +
                            scale_color_brewer(palette = "Set1") +
                            xlab(NULL) +
                            ylab(expression(paste("Discharge (",m^3," ",s^-1,")"))) +
                            theme_bw() +
                            theme(legend.title = element_blank(),
                                  text=element_text(size=18))

ggsave(file=file.path(output_dir,"Hydrographs",paste(dataSetName,"_Baseflows.jpeg")),
       baseflowHydrograph,
       width = 14, 
       height = 4, 
       units = "in",
       dpi=600)


# 3) Plot smoothed storm flows

stormflowHydrograph <- ggplot() + 
  geom_line(data=batchRunFlowsLF1, aes(x=datetime, y=storm_flow,color=filter_para), size=0.75) +
  scale_color_brewer(palette = "Set1") +
  xlab(NULL) +
  ylab(expression(paste("Storm flow (",m^3," ",s^-1,")"))) +
  theme_bw() +
  theme(legend.title = element_blank(),
        text=element_text(size=18))

ggsave(file=file.path(output_dir,"Hydrographs",paste(dataSetName,"_StormflowsOnly.jpeg")),
       stormflowHydrograph,
       width = 14, 
       height = 4, 
       units = "in",
       dpi=600)


# 3a) Plot smoothed storm flows with storm flow thresholds

stormflowThreshHydrograph <- ggplot() + 
  geom_line(data=batchRunFlowsLF1, aes(x=datetime, y=storm_flow,color=filter_para), size=0.75) +
  scale_color_brewer(palette = "Set1") +
  geom_hline(yintercept = candidateSfThresh, linetype = "dashed", color = "black",alpha=0.5) +
  xlab(NULL) +
  ylab(expression(paste("Storm flow (",m^3," ",s^-1,")"))) +
  scale_x_datetime(limits = as.POSIXct(c("2020-04-01 00:00:00", "2020-07-01 01:00:00"))) + # MED added for comparison to HydRun results
  theme_bw() +
  theme(legend.title = element_blank(),
        text=element_text(size=18))

ggsave(file=file.path(output_dir,"Hydrographs",paste(dataSetName,"_StormflowsOnlyWithThresholds.jpeg")),
       stormflowThreshHydrograph,
       width = 14, 
       height = 4, 
       units = "in",
       dpi=600)


# 4) Plot batch run event separation hydrographs
eventsDataShaded1 <- eventsData1 %>% mutate(start = as.POSIXct(start,
                                                                   format("%Y-%m-%d %H:%M:%S"),tz="EST"),
                                            end = as.POSIXct(end,
                                                                   format("%Y-%m-%d %H:%M:%S"),tz="EST"),
                                            tops = max(Site_input$q_cms),
                                            bottoms = 0)

batchEventSepPlot <- ggplot() + 
  geom_rect(data=eventsDataShaded1, mapping=aes(xmin=start, 
                                                xmax=end, 
                                                ymin=bottoms, 
                                                ymax=tops), fill="green", color="red", alpha=0.2) +
  
  geom_line(data=Site_input, aes(x=datetime, y=q_cms), size=0.8, color="blue") +
  geom_line(data=Site_input, aes(x=datetime, y=rescaled_conc), size=0.5, color="black",linetype="dashed") +
  facet_wrap(~ run_id, ncol = 1) +
  scale_color_brewer(palette = "Set1") +
  xlab(NULL) +
  ylab(expression(paste("Discharge (",m^3," ",s^-1,")"))) +
  scale_x_datetime(limits = as.POSIXct(c("2023-02-05 00:00:00", "2023-02-18 01:00:00"))) + # MED added
  theme_bw() +
  theme(legend.title = element_blank(),
        text=element_text(size=18))


ggsave(file=file.path(output_dir,"Hydrographs",paste(dataSetName,"_BatchEventSeparationPlot.jpeg")),
       batchEventSepPlot,
       width = 14, # 8 original MED changed
       height = 4, # 10 original MED changed
       units = "in",
       dpi=600)

Warning message:
“Removed 682 rows containing missing values (`geom_line()`).”
Warning message:
“Removed 3 rows containing missing values (`geom_rect()`).”


In [43]:
####################################
# PLOT AND SAVE DATA - c-Q RESULTS #
####################################

if (constit == "NO3") {
  
  makeCQPlotsNO3(batchRun1)
  makeHystFlushPlotsNO3(hysteresisData1)
  
} else if (constit == "TOC") {

makeCQPlotsTOC(batchRun1)
makeHystFlushPlotsTOC(hysteresisData1)
  
} else if (constit == "turbity") {
    
makeCQPlotsTurb(batchRun1) 
makeHystFlushPlotsTurb(hysteresisData1)
  
}